# Libraries

In [8]:
import random
import pandas as pd
from typing import Dict, List, Optional
from dataclasses import dataclass
import logging
from datetime import datetime

## Players

In [9]:
@dataclass
class Player:
    # Initial data (from excel file)
    name: str
    team: str
    role: str  # 'GK', 'DEF', 'MID', 'ATT'
    evaluation: int

    ## Calculated after the importing of the file
    standardized_evaluation: float = 0.0  # Between 0 and 1
    ranking: int = 0  # Rank based on evaluation
    
    # Final results (after auction)
    fantasy_team: Optional[str] = None  # Which agent bought the player
    final_cost: int = None  # Final price paid

def calculate_standardized_and_ranking(players: List[Player]) -> List[Player]:
    """Calculate standardized evaluation and ranking for all players"""
    # Sort players by evaluation (highest first)
    sorted_players = sorted(players, key=lambda p: p.evaluation, reverse=True)
    
    # Get min and max evaluations for standardization
    max_eval = max(p.evaluation for p in players)
    min_eval = min(p.evaluation for p in players)
    eval_range = max_eval - min_eval
    
    # Assign rankings and standardized evaluations
    for i, player in enumerate(sorted_players):
        player.ranking = i + 1  # Ranking starts from 1
        # Standardize: (value - min) / (max - min)
        if eval_range > 0:
            player.standardized_evaluation = (player.evaluation - min_eval) / eval_range
        else:
            player.standardized_evaluation = 0.5  # If all players have same evaluation
    
    return players

## Import players from list
# Read Excel file
df = pd.read_excel("data/players_list.xlsx")

# Create Player objects
players = []
for _, row in df.iterrows():
    player = Player(
        name=row['name'],
        team=row['team'], 
        role=row['role'],
        evaluation=row['evaluation']
    )
    players.append(player)

# Calculate standardized evaluation and ranking
players = calculate_standardized_and_ranking(players)

print(players)

[Player(name='Sommer', team='Inter', role='GK', evaluation=90, standardized_evaluation=0.23482849604221637, ranking=38, fantasy_team=None, final_cost=None), Player(name='Di Gregorio', team='Juventus', role='GK', evaluation=83, standardized_evaluation=0.21635883905013192, ranking=41, fantasy_team=None, final_cost=None), Player(name='Maignan', team='Milan', role='GK', evaluation=81, standardized_evaluation=0.21108179419525067, ranking=43, fantasy_team=None, final_cost=None), Player(name='Meret', team='Napoli', role='GK', evaluation=92, standardized_evaluation=0.24010554089709762, ranking=36, fantasy_team=None, final_cost=None), Player(name='Svilar', team='Roma', role='GK', evaluation=82, standardized_evaluation=0.21372031662269128, ranking=42, fantasy_team=None, final_cost=None), Player(name='Carnesecchi', team='Atalanta', role='GK', evaluation=71, standardized_evaluation=0.18469656992084432, ranking=48, fantasy_team=None, final_cost=None), Player(name='De Gea', team='Fiorentina', role='

## Rules

In [10]:
class Slots:
    def __init__(self, gk: int = 3, def_: int = 8, mid: int = 8, att: int = 6):
        self.gk = gk
        self.def_ = def_
        self.mid = mid
        self.att = att
        
    def to_dict(self) -> Dict[str, int]:
        return {'GK': self.gk, 'DEF': self.def_, 'MID': self.mid, 'ATT': self.att}

## Agents

In [11]:
class Squad:
    def __init__(self, players: List['Player']):
        self._players = players

    def __iter__(self):
        return iter(self._players)

    def __len__(self):
        return len(self._players)

    def __getitem__(self, idx):
        return self._players[idx]

    @property
    def gk(self):
        return [p for p in self._players if p.role == "GK"]

    @property
    def def_(self):
        return [p for p in self._players if p.role == "DEF"]

    @property
    def mid(self):
        return [p for p in self._players if p.role == "MID"]

    @property
    def att(self):
        return [p for p in self._players if p.role == "ATT"]
    
    def objective(self, bestxi: bool = False, standardized: bool = False) -> float:
        """
        Calculate objective function for this squad
        
        Args:
            bestxi: If True, calculate only best starting XI (1 GK + 4 DEF + 4 MID + 3 ATT)
                   If False, calculate total squad evaluation
            standardized: If True, use standardized_evaluation
                         If False, use regular evaluation
        
        Returns:
            float: The calculated objective value
        """
        eval_attr = 'standardized_evaluation' if standardized else 'evaluation'
        
        if bestxi:
            # Best starting XI: 1 GK + 4 DEF + 4 MID + 3 ATT
            best_gk = max(self.gk, key=lambda p: getattr(p, eval_attr), default=None)
            best_def = sorted(self.def_, key=lambda p: getattr(p, eval_attr), reverse=True)[:4]
            best_mid = sorted(self.mid, key=lambda p: getattr(p, eval_attr), reverse=True)[:4]
            best_att = sorted(self.att, key=lambda p: getattr(p, eval_attr), reverse=True)[:3]
            
            total = 0
            if best_gk:
                total += getattr(best_gk, eval_attr)
            total += sum(getattr(p, eval_attr) for p in best_def + best_mid + best_att)
            
            return total
        else:
            # Total squad evaluation
            return sum(getattr(p, eval_attr) for p in self._players)

class Agent:
    def __init__(self, agent_id: str):
        self.agent_id = agent_id
        self.current_credits = 1000  # Default, will be set properly in initialize
        self._squad: List[Player] = []

    @property
    def squad(self):
        return Squad(self._squad)
    
    def initialize(self, listone: List[Player], slots: Slots, initial_credits: int, num_participants: int):
        """Initialize agent with auction environment - to be overridden by subclasses if needed"""
        self.current_credits = initial_credits

    def make_offer_decision(self, current_player, current_price, highest_bidder, player_list) -> str:
        raise NotImplementedError("This method should be implemented by subclasses.")

class RandomAgent(Agent):
    def make_offer_decision(self, current_player, current_price, highest_bidder, player_list) -> str:
        return "offer_+1" if random.random() < 0.5 else "no_offer"

class CapAgent(Agent):
    def __init__(self, agent_id: str, cap_strategy: str = "value_based", bestxi_budget: float = 0.9):
        super().__init__(agent_id)
        self.cap_strategy = cap_strategy
        self.bestxi_budget = bestxi_budget
        self.listone = {}  # Will be filled during initialize
        self.players_caps = {}  # Dictionary to store player caps
    
    def initialize(self, listone: List[Player], slots: Slots, initial_credits: int, num_participants: int):
        """Initialize agent with auction environment"""
        super().initialize(listone, slots, initial_credits, num_participants)

        if self.cap_strategy == "value_based":

            # Calculate average budget per team for the best players 
            avg_budget_per_team = initial_credits * self.bestxi_budget

            # Filter best players by role based on actual slot requirements:
            best_gk = sorted([p for p in listone if p.role == "GK"], 
                           key=lambda p: p.evaluation, reverse=True)[:1 * num_participants]
            best_def = sorted([p for p in listone if p.role == "DEF"], 
                            key=lambda p: p.evaluation, reverse=True)[:3 * num_participants]
            best_mid = sorted([p for p in listone if p.role == "MID"], 
                            key=lambda p: p.evaluation, reverse=True)[:3 * num_participants]
            best_att = sorted([p for p in listone if p.role == "ATT"], 
                            key=lambda p: p.evaluation, reverse=True)[:3 * num_participants]
            
            # Combine all best players
            best_players = best_gk + best_def + best_mid + best_att

            # Calculate total evaluation points available
            total_evaluation = sum(p.evaluation for p in best_players)

            # Calculate value per evaluation point
            value_per_point = (num_participants * avg_budget_per_team) / total_evaluation

            # For ONLY the best players, compute cap and store in dictionary
            for player in best_players:
                cap = player.evaluation * value_per_point
                self.players_caps[player.name] = cap


            # Calculate average budget per team for the acceptable players (10% of total credits)
            avg_budget_per_team = initial_credits * (1 - self.bestxi_budget)

            # Filter acceptable players by role based on actual slot requirements:
            acceptable_gk = sorted([p for p in listone if p.role == "GK"], 
                           key=lambda p: p.evaluation, reverse=True)[:slots.gk * num_participants]
            acceptable_def = sorted([p for p in listone if p.role == "DEF"], 
                            key=lambda p: p.evaluation, reverse=True)[:slots.def_ * num_participants]
            acceptable_mid = sorted([p for p in listone if p.role == "MID"], 
                            key=lambda p: p.evaluation, reverse=True)[:slots.mid * num_participants]
            acceptable_att = sorted([p for p in listone if p.role == "ATT"], 
                            key=lambda p: p.evaluation, reverse=True)[:slots.att * num_participants]
            
            # Combine all accptable players and remove the best
            acceptable_players = acceptable_gk + acceptable_def + acceptable_mid + acceptable_att 
            acceptable_players = [p for p in acceptable_players if p not in best_players] 

            # Calculate total evaluation points available
            total_evaluation = sum(p.evaluation for p in acceptable_players)

            # Calculate value per evaluation point
            value_per_point = (num_participants * avg_budget_per_team) / total_evaluation

            # For ONLY the average players, compute cap and store in dictionary
            for player in acceptable_players:
                cap = player.evaluation * value_per_point
                self.players_caps[player.name] = cap
    
    def make_offer_decision(self, current_player, current_price, highest_bidder, player_list) -> str:
        """Make bidding decision based on cap calculation"""

        if current_player.name not in self.players_caps:
            return "offer_+1" if random.random() < 0.01 else "no_offer"
        
        max_bid = self.players_caps[current_player.name]
    
        # Don't bid if current price exceeds our calculated maximum
        if current_price >= max_bid:
            return "no_offer"
        else:
            return "offer_+1" 

## Auction Class

In [12]:
class Auction:
    def __init__(self, slots: Slots, agents: List[Agent], players: List[Player] = players, initial_credits: int = 1000,):
        ## Rules
        self.slots = slots.to_dict()
        self.initial_credits = initial_credits

        ## Rest
        self.agents = agents
        self.players = players

        # Initialize ALL agents with auction variables
        for agent in self.agents:
            agent.initialize(players, slots, initial_credits, len(agents))
                
    def can_participate_in_bid(self, agent: Agent, current_price: int, position: str, slots: Dict[str, int]) -> bool:
        """
        Check if the agent is not the highest bidder, has a free slot for the given position
        and can afford to buy the player while still keeping enough credits
        to fill the rest of their squad.
        """
        
        # Check if they have at least one free slot in this position
        role_map = {"GK": agent.squad.gk, "DEF": agent.squad.def_, "MID": agent.squad.mid, "ATT": agent.squad.att}
        if len(role_map[position]) >= slots[position]:
            return False
        
        # Calculate credits left after the purchase (current_price + 1 is what they would pay)
        credits_after_purchase = agent.current_credits - (current_price + 1)

        # Calculate total max slots in the squad (all positions)
        total_slots = sum(slots.values())

        # Calculate total players currently in the squad (all positions)
        total_players = len(agent.squad)

        # Since we are bidding for a player in 'position', this bid would fill one slot
        remaining_slots = total_slots - (total_players + 1)

        # They can participate only if remaining credits cover minimum needed
        return credits_after_purchase >= remaining_slots
        
    def single_player(self, player: Player): 
        """Auction for a single player"""
        
        # Set initial state
        self.current_player = player
        self.current_price = 0  # Start at 0
        self.highest_bidder = None
        
        agent_index = 0  # Track which agent's turn it is
        no_bid_count = 0  # Count consecutive agents who didn't bid
        
        # Log auction start
        logger = logging.getLogger()
        logger.info(f"\n🏆 AUCTION START: {player.name} ({player.role}) - Evaluation: {player.evaluation}")
        
        while True:
            current_agent = self.agents[agent_index]
            
            # Check if agent can participate
            if (self.can_participate_in_bid(agent= current_agent, current_price = self.current_price, position = player.role, slots= self.slots)):
                
                # Agent makes decision
                decision = current_agent.make_offer_decision(
                    current_player=player,
                    current_price=self.current_price,
                    highest_bidder=self.highest_bidder,
                    player_list=self.players
                )
                
                if decision == "offer_+1":
                    self.current_price += 1
                    self.highest_bidder = current_agent.agent_id
                    no_bid_count = 0  # Reset counter
                    
                    # Log the bid
                    logger.info(f"  💰 {current_agent.agent_id} bids {self.current_price} (credits left: {current_agent.current_credits})")
                else:
                    no_bid_count += 1
                    
                    # Log the pass
                    logger.info(f"  ❌ {current_agent.agent_id} passes at {self.current_price}")
            else:
                no_bid_count += 1
                
                # Log why agent can't participate
                logger.info(f"  🚫 {current_agent.agent_id} cannot bid (no slot/credits)")
            
            # Move to next agent
            agent_index = (agent_index + 1) % len(self.agents)
            
            # Check if full round completed with no bids
            if no_bid_count >= (len(self.agents)-1):  # -1 because highest bidder is excluded
                break
        
        # Finalize sale
        if self.highest_bidder: #if the highest bid exists
            
            player.fantasy_team = self.highest_bidder
            player.final_cost = self.current_price
            
            # Add player to agent's squad and deduct credits 
            winner_agent = next(agent for agent in self.agents if agent.agent_id == self.highest_bidder)
            winner_agent._squad.append(player)
            winner_agent.current_credits -= self.current_price
            
            logger.info(f"✅ {player.name} SOLD to {self.highest_bidder} for {self.current_price}. Credits remaining: {winner_agent.current_credits}")
            return self.highest_bidder
        else:
            player.fantasy_team = "UNSOLD"  # Mark as unsold instead of None
            player.final_cost = 0
            logger.info(f"❌ {player.name} UNSOLD")
            return None
        
    def run_all(self, auction_type: str = "random", per_ruolo: bool = True):
        """Run auction for all players, role by role if per_ruolo is True.
        Only auction players not already assigned to a fantasy team.
        Returns the updated list of players.
        """
        import logging
        logger = logging.getLogger()
        
        roles = ["GK", "DEF", "MID", "ATT"]
        self.auction_type = auction_type
        self.per_ruolo = per_ruolo

        players_to_auction = self.players[:]
        if auction_type == "random":
            random.shuffle(players_to_auction)
        elif auction_type == "listone":
            players_to_auction.sort(key=lambda p: p.name)
        elif auction_type == "chiamata":
            players_to_auction.sort(key=lambda p: p.evaluation, reverse=True)

        if per_ruolo:
            for role in roles:
                logger.info(f"\n{'='*50}\n🏈 STARTING {role} AUCTIONS\n{'='*50}")
                    
                while True:
                    # Check if all agents have filled their slots for this role
                    all_full = all(len(getattr(agent.squad, role.lower() if role != "DEF" else "def_")) >= self.slots[role] for agent in self.agents)
                    if all_full:
                        break
                    # Find next available player for this role (exclude unsold players)
                    next_player = next((p for p in players_to_auction if p.role == role and p.fantasy_team is None), None)
                    if next_player is None:
                        break  # No more available players for this role
                    self.single_player(next_player)
        else:
            total_slots = sum(self.slots.values())
            while True:
                # Check if all agents have full squads
                all_full = all(len(agent.squad) >= total_slots for agent in self.agents)
                if all_full:
                    break
                # Find next available player (exclude unsold players)
                next_player = next((p for p in players_to_auction if p.fantasy_team is None), None)
                if next_player is None:
                    break  # No more available players
                self.single_player(next_player)

        return None

## single player trial

In [13]:
# Create auction with 4 agents
slots = Slots()
initial_credits = 1000
agents = [
    CapAgent(agent_id="cucco"),
    CapAgent(agent_id="andrea"),
    RandomAgent(agent_id="gianmarco"),
    RandomAgent(agent_id="carlomario")
]  

auction = Auction(slots, agents, players)
haaland = next(p for p in players if p.name == "Sommer")
auction.single_player(haaland)
print(agents[1].players_caps)



🏆 AUCTION START: Sommer (GK) - Evaluation: 90
  💰 cucco bids 1 (credits left: 1000)
  💰 andrea bids 2 (credits left: 1000)
  ❌ gianmarco passes at 2
  ❌ carlomario passes at 2
  💰 cucco bids 3 (credits left: 1000)
  💰 andrea bids 4 (credits left: 1000)
  💰 gianmarco bids 5 (credits left: 1000)
  💰 carlomario bids 6 (credits left: 1000)
  💰 cucco bids 7 (credits left: 1000)
  💰 andrea bids 8 (credits left: 1000)
  💰 gianmarco bids 9 (credits left: 1000)
  ❌ carlomario passes at 9
  💰 cucco bids 10 (credits left: 1000)
  💰 andrea bids 11 (credits left: 1000)
  💰 gianmarco bids 12 (credits left: 1000)
  💰 carlomario bids 13 (credits left: 1000)
  💰 cucco bids 14 (credits left: 1000)
  💰 andrea bids 15 (credits left: 1000)
  💰 gianmarco bids 16 (credits left: 1000)
  💰 carlomario bids 17 (credits left: 1000)
  💰 cucco bids 18 (credits left: 1000)
  💰 andrea bids 19 (credits left: 1000)
  ❌ gianmarco passes at 19
  💰 carlomario bids 20 (credits left: 1000)
  💰 cucco bids 21 (credits left: 

{'Meret': 53.41074020319303, 'Sommer': 52.24963715529753, 'Di Gregorio': 48.185776487663276, 'Svilar': 47.60522496371553, 'Dumfries': 58.055152394775035, 'Dimarco': 55.73294629898403, 'Gosens': 49.92743105950653, 'Bastoni': 45.283018867924525, 'Zortea': 39.477503628447025, 'Cambiaso': 36.574746008708274, 'Angelino': 34.25253991291727, 'Di Lorenzo': 33.09143686502177, 'Zappacosta': 30.769230769230766, 'Wesley': 30.18867924528302, 'Bellanova': 30.18867924528302, 'Tavares N.': 28.447024673439767, 'Orsolini': 127.72133526850507, 'Pulisic': 125.39912917271407, 'Zaccagni': 117.85195936139331, 'McTominay': 116.69085631349782, 'Gudmundsson A.': 102.75761973875181, 'Calhanoglu': 88.24383164005805, 'De Bruyne': 85.3410740203193, 'Paz N.': 69.08563134978229, 'Conceicao': 52.83018867924528, 'Odgaard': 45.283018867924525, 'Thuram K.': 37.73584905660377, 'Ederson D.S.': 33.67198838896952, 'Martinez L.': 220.60957910014514, 'Kean': 217.70682148040638, 'Lukaku': 197.96806966618286, 'Thuram': 184.61538

## whole auction trial

In [ ]:
if __name__ == "__main__":
    
    # Set up logging - clear existing handlers first
    logger = logging.getLogger()
    for handler in logger.handlers[:]:
        logger.removeHandler(handler)
    
    log_filename = f"auction_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
    logging.basicConfig(
        level=logging.INFO,
        format='%(message)s',
        handlers=[
            logging.FileHandler(log_filename, encoding='utf-8'),
            logging.StreamHandler()  # This still prints to console too
        ],
        force=True  # Force reconfiguration
    )
    logger = logging.getLogger()
    
    # Create the characteristics of the auction
    slots = Slots()
    initial_credits = 1000
    agents = [
        CapAgent(agent_id="cap agent 1", bestxi_budget= 1.2),
        CapAgent(agent_id="cap agent 2",  bestxi_budget= 1),
        CapAgent(agent_id="cap agent 3",  bestxi_budget= 0.9),
        CapAgent(agent_id="cap agent 4",  bestxi_budget= 0.8),
        RandomAgent(agent_id="davide"),
        RandomAgent(agent_id="tommi"),
        RandomAgent(agent_id="gianmarco"),
        RandomAgent(agent_id="carlomario")
    ]  
    auction_type = "random"
    per_ruolo = True
    auction = Auction(slots, agents)

    # Log auction and agent configuration
    logger.info("\n⚙️ AUCTION CONFIGURATION")
    logger.info("-" * 40)
    logger.info(f"📊 Slots per team:")
    logger.info(f"  GK: {slots.gk}")
    logger.info(f"  DEF: {slots.def_}")
    logger.info(f"  MID: {slots.mid}")
    logger.info(f"  ATT: {slots.att}")
    logger.info(f"💰 Initial credits per agent: {initial_credits}")
    logger.info(f"👥 Number of agents: {len(agents)}")
    logger.info(f"📋 Total players available: {len(players)}")
    logger.info(f"  Auction type: {auction_type}")
    logger.info(f"  Per role: {per_ruolo}")    
    logger.info("\n🤖 AGENT CONFIGURATIONS")
    logger.info("-" * 40)
    for agent in agents:
        if isinstance(agent, CapAgent):
            logger.info(f"  {agent.agent_id}: CapAgent (strategy: {agent.cap_strategy})")
        elif isinstance(agent, RandomAgent):
            logger.info(f"  {agent.agent_id}: RandomAgent")
        else:
            logger.info(f"  {agent.agent_id}: {type(agent).__name__}")


    logger.info("🏁 AUCTION SIMULATION STARTED")
    logger.info(f"📅 Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    logger.info("="*60)
    logger.info("\n" + "="*60)

    # Run auction
    auction.run_all(auction_type=auction_type, per_ruolo=per_ruolo)
    

    logger.info("🏁 AUCTION COMPLETED!")
    logger.info(f"📅 Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    logger.info("="*60)
    
    # Log all team information instead of creating Excel files
    logger.info("\n" + "="*80)
    logger.info("📊 FINAL TEAMS BY AGENT")
    logger.info("="*80)
    
    for agent in auction.agents:
        logger.info(f"\n🏆 TEAM: {agent.agent_id.upper()}")
        logger.info("-" * 50)
        
        # Log squad players
        for player in agent.squad:
            logger.info(f"  {player.role:3} | {player.name:20} | {player.team:15} | Eval: {player.evaluation:3} | Cost: {player.final_cost:3}")
        
        # Calculate squad objectives
        total_eval = agent.squad.objective()
        total_std_eval = agent.squad.objective(standardized=True)
        bestxi_eval = agent.squad.objective(bestxi=True)
        bestxi_std_eval = agent.squad.objective(bestxi=True, standardized=True)
        
        # Log summary metrics
        logger.info("\n📈 SQUAD METRICS:")
        logger.info(f"  Total Squad Evaluation: {total_eval}")
        logger.info(f"  Total Squad Standardized: {total_std_eval:.3f}")
        logger.info(f"  Best XI Evaluation: {bestxi_eval}")
        logger.info(f"  Best XI Standardized: {bestxi_std_eval:.3f}")
        logger.info(f"  Credits Remaining: {agent.current_credits}")
        logger.info(f"  Total Credits Spent: {1000 - agent.current_credits}")
        logger.info("")
        logger.info("="*60)

    
    for agent in auction.agents:
        logger.info(f"\n🏆 TEAM: {agent.agent_id.upper()}")
        logger.info("-" * 50)
                # Calculate squad objectives
        total_eval = agent.squad.objective()
        total_std_eval = agent.squad.objective(standardized=True)
        bestxi_eval = agent.squad.objective(bestxi=True)
        bestxi_std_eval = agent.squad.objective(bestxi=True, standardized=True)
                # Log summary metrics
        logger.info("\n📈 SQUAD METRICS:")
        logger.info(f"  Total Squad Evaluation: {total_eval}")
        logger.info(f"  Total Squad Standardized: {total_std_eval:.3f}")
        logger.info(f"  Best XI Evaluation: {bestxi_eval}")
        logger.info(f"  Best XI Standardized: {bestxi_std_eval:.3f}")
        logger.info(f"  Credits Remaining: {agent.current_credits}")
        logger.info(f"  Total Credits Spent: {1000 - agent.current_credits}")
        logger.info("")

    logger.info(f"\n📁 Complete log saved to: {log_filename}")


⚙️ AUCTION CONFIGURATION
----------------------------------------
📊 Slots per team:
  GK: 3
  DEF: 8
  MID: 8
  ATT: 6
💰 Initial credits per agent: 1000
👥 Number of agents: 8
📋 Total players available: 513
  Auction type: random
  Per role: True

🤖 AGENT CONFIGURATIONS
----------------------------------------
  cap agent 1: CapAgent (strategy: value_based)
  cap agent 2: CapAgent (strategy: value_based)
  cap agent 3: CapAgent (strategy: value_based)
  cap agent 4: CapAgent (strategy: value_based)
  davide: RandomAgent
  tommi: RandomAgent
  gianmarco: RandomAgent
  carlomario: RandomAgent
🏁 AUCTION SIMULATION STARTED
📅 Timestamp: 2025-08-02 22:59:08


🏈 STARTING GK AUCTIONS

🏆 AUCTION START: Sommariva (GK) - Evaluation: 1
  ❌ cap agent 1 passes at 0
  ❌ cap agent 2 passes at 0
  ❌ cap agent 3 passes at 0
  ❌ cap agent 4 passes at 0
  💰 davide bids 1 (credits left: 1000)
  💰 tommi bids 2 (credits left: 1000)
  ❌ gianmarco passes at 2
  ❌ carlomario passes at 2
  ❌ cap agent 1 passes a